# CoursewareHubの構築時の問題分析のための情報収集

---

CoursewareHubの構築時に発生した問題を分析するための情報を収集します。

以下のNotebookで構築する内容を対象とします。
- 011-VCノード作成-構成1.ipynb
- 021-VCノード作成-構成2.ipynb
- 031-VCノード作成-構成3.ipynb
- 121-CoursewareHubのセットアップ-ローカルユーザ認証.ipynb
- 221-CoursewareHubのセットアップ.ipynb
- 321-CoursewareHubのセットアップ.ipynb

## はじめに



`000-README.ipynb`の`5 作業用Notebookの作成`で指定した`作業用Notebookを配置するディレクトリ`にこのNotebookを配置し実行ください。
他のディレクトリに配置した場合正しく動作しません。

ここでは以降の情報収集準備として、対象となるUnitGroup等の指定と設定済みパラメータの読み込みを実施します。
また、VCCにアクセスするために必要となるアクセストークンの入力を行います。

入力すべき項目は全てこの章にまとまっていますので、この章を実行後は以降の章についてはまとめて実行いただいて問題ございません。

group_varsファイル名の一覧を表示します。group_varsへの保存が実施されていた場合はUnitGeroup名で設定ファイルが作成されています。

In [ ]:
!ls -1 group_vars/

上記の出力も参考に情報収集の対象となるUnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

manager unit名とworker unit名を指定します。デフォルトから変更してしていない場合はそのまま実行ください。

In [ ]:
manager_name = 'manager'
worker_name = 'worker'

NFS server用 unit名を指定してください。managerノードにNFSサーバを配置する構成としている場合やデフォルトから変更してしていない場合はそのまま実行ください。

In [ ]:
nfs_name = 'nfs' #managerとNFSサーバを別々のノードに分離した場合のデフォルト名

設定しているパラメータの情報を読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

設定しているパラメータからNFSサーバを操作するための Ansible のグループ名を設定します。次のセルを実行ください。

In [ ]:
if 'nfs_target' in gvars:
    nfs_target = gvars['nfs_target']
elif 'nfs_flavor' in gvars:
    nfs_target = f'{ugroup_name}_{nfs_name}'
else:
    nfs_target = f'{ugroup_name}_{manager_name}'

print(nfs_target)

VCノード等の情報を収集するためにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合は、もう一度アクセストークンの入力のセルを`unfreeze` し、アクセストークンの入力をやり直してください。

## パラメータの設定の確認

テンプレートでは、パラメータの保存形式は `Ansible` のフォーマットに従い、`group_vars/` のYAMLファイルとして保存します。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}

デフォルトパラメータの内容を表示します。

In [ ]:
!cat group_vars/all/defaults.yml

## UnitGroup、VCノード等の確認
以降の章では次のNotebookで構築する内容を確認します。
- 011-VCノード作成-構成1.ipynb
- 021-VCノード作成-構成2.ipynb
- 031-VCノード作成-構成3.ipynb

### VCPバージョンの確認

In [ ]:
vcp.version()

### 作成済みのUnitGroup、Unit、VCノードの確認

#### UnitGroup一覧

In [ ]:
vcp.df_ugroups()

#### 情報収集対象のUnitGroupのUnitとNodeの一覧

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_units()


In [ ]:
ug.df_nodes()

## Ansibleの確認


### インベントリファイルの確認

次のセルを実行すると作成したインベントリの内容を表示します。

In [ ]:
!cat inventory.yml

###  コンフィギュレーションファイルの確認


次のセルを実行すると作成したコンフィギュレーションファイルの内容を表示します。

In [ ]:
!cat ansible.cfg

### Ansibleを使った登録済みノードへの接続の確認
UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。テンプレートではグループ名は UnitGroup名と同じ値にしてあります。

In [ ]:
!ansible {ugroup_name} -m ping

## NFSの確認


### NFSサーバの確認

NFSサーバのサービス状態を確認します。

In [ ]:
!ansible {nfs_target} -b -a 'systemctl status nfs-server'

NFSエクスポートの状態を確認します。テンプレートではmanagerノードの起動時は全てのノードに対してアクセス可能な状態に設定されています。その後workerノードを起動してNFSクライアントのIPアドレスが確定した時点で`/etc/exports`の設定を更新し、アクセスできるノードの制限を行います。

In [ ]:
!ansible {nfs_target} -b -a 'exportfs -v'

ディレクトリのパーミッションを設定します。
テンプレートでは777が設定されます。

In [ ]:
if 'nfs_target' in gvars:
    !ansible {nfs_target} -b -a 'ls -al /exported/{ugroup_name}'
else:
    !ansible {nfs_target} -b -a 'ls -al /exchange'
    !ansible {nfs_target} -b -a 'ls -al /jupyter'

### managerノードでのNFSマウント状況の出力
managerノードにNFSサーバを配置する構成の場合はスキップしてください。

managerノードにおけるNFSマウントの状態を確認します。`/exchange`, `/jupyter`のエントリが存在していることを確認してください。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'mount -t nfs4'

NFSサーバ側に作成したファイルをNFSクライアント側で参照できることを確認します。

In [ ]:
if 'nfs_target' in gvars:
    !ansible {nfs_target} -m file -a 'path=/exported/{ugroup_name}/jupyter/xxx state=touch'
    !ansible {nfs_target} -m file -a 'path=/exported/{ugroup_name}/exchange/xxx state=touch'
else:
    !ansible {nfs_target} -m file -a 'path=/jupyter/xxx state=touch'
    !ansible {nfs_target} -m file -a 'path=/exchange/xxx state=touch'

!ansible {ugroup_name}_{manager_name} -a 'test -f /jupyter/xxx'
!ansible {ugroup_name}_{manager_name} -a 'test -f /exchange/xxx'
!ansible {ugroup_name} -m file -a 'path=/jupyter/xxx state=absent'
!ansible {ugroup_name} -m file -a 'path=/exchange/xxx state=absent'

### workerノードでのNFSマウント状況の出力

workerノードにおけるNFSマウントの状態を確認します。`/exchange`, `/jupyter`のエントリが存在していることを確認してください。

In [ ]:
!ansible {ugroup_name}_{worker_name} -a 'mount -t nfs4'

NFSサーバ側に作成したファイルをNFSクライアント側で参照できることを確認します。

In [ ]:
if 'nfs_target' in gvars:
    !ansible {nfs_target} -m file -a 'path=/exported/{ugroup_name}/jupyter/xxx state=touch'
    !ansible {nfs_target} -m file -a 'path=/exported/{ugroup_name}/exchange/xxx state=touch'
else:
    !ansible {nfs_target} -m file -a 'path=/jupyter/xxx state=touch'
    !ansible {nfs_target} -m file -a 'path=/exchange/xxx state=touch'

!ansible {ugroup_name}_{worker_name} -a 'test -f /jupyter/xxx'
!ansible {ugroup_name}_{worker_name} -a 'test -f /exchange/xxx'
!ansible {ugroup_name} -m file -a 'path=/jupyter/xxx state=absent'
!ansible {ugroup_name} -m file -a 'path=/exchange/xxx state=absent'

## Docker Swarmの設定の確認



各VCノードで Docker Engine が実行されていることを確認します。

In [ ]:
!ansible {ugroup_name} -a 'docker info'

Docker Swarmのノード一覧を表示します。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'docker node ls'

In [ ]:
!ansible {ugroup_name}_{manager_name} -m shell -a \
    'docker node ls -q'

Docker Swarmのノード数が起動したVCノードと一致していることを確認します。

In [ ]:
worker_nodes = gvars['worker_nodes']
!ansible {ugroup_name}_{manager_name} -m shell -a \
    'test $(docker node ls -q | wc -l) -eq {worker_nodes + 1}'

## CoursewareHubのセットアップパラメータの確認
以降の章では次のNotebookで構築する内容を確認します。
- 121-CoursewareHubのセットアップ-ローカルユーザ認証.ipynb
- 221-CoursewareHubのセットアップ.ipynb
- 321-CoursewareHubのセットアップ.ipynb


### リソース制限の設定を行うYAMLファイルの確認

作成したファイルの内容を表示します。

In [ ]:
rsc_yml = gvars['rsc_yml']
!cat {rsc_yml}

YAMLファイルの記述内容が妥当であるかをチェックします。

次のセルが正常に実行できることを確認してください。実行結果がエラーとなった場合はYAMLファイルの記述内容に問題があります。

In [ ]:
import jsonschema
import json
import yaml
import os

rsc_dir = Path(os.path.dirname(rsc_yml))

!ansible -c local {ugroup_name}_{manager_name} -m get_url -a \
    'url=https://raw.githubusercontent.com/NII-cloud-operation/CoursewareHub-LC_platform/master/jupyterhub/resources-schema.json \
    dest={rsc_dir}/resources-schema.json'
with (rsc_dir / 'resources-schema.json').open() as f:
    resources_config_schema = json.load(f)

with (rsc_dir / 'resource.yaml').open() as f:
    resources_config = yaml.load(f, Loader=yaml.SafeLoader)
    jsonschema.validate(resources_config, resources_config_schema)

print(json.dumps(resources_config, indent=2))

managerノード上に配置されたファイルの内容を表示します。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'cat {{{{jupyterhub_dir}}}}/resource.yaml'

### オーバーレイネットワークの指定の確認

指定されたサブネットがVCノードに割り当てられているIPアドレスと重なっていないことをチェックします。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
cousewarehub_backend = gvars['cousewarehub_backend']
!ansible-playbook -v -e cousewarehub_backend={cousewarehub_backend} -l {ugroup_name} \
    playbooks/check-subnet.yml

## CoursewareHubのセットアップの確認

### auth-proxyの確認

証明書が配置されていることを確認します。managerノードのサーバ証明書の内容を表示します。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a \
    'openssl x509 -noout -text -in {{{{certs_dir}}}}/auth-proxy.cer'

秘密鍵の内容を表示します。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a \
    'sudo openssl rsa -noout -text -in  {{{{certs_dir}}}}/auth-proxy.key'

中間CA証明書を連結したサーバ証明書の内容を表示します。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a \
    'openssl x509 -noout -text -in {{{{certs_dir}}}}/auth-proxy.chained.cer'

直接学認フェデレーションを利用する場合は配置したメタデータ署名証明書のFingerprintを表示します。

参考のため、証明書の fingerprintを記載しているURLを以下に示します。

* 運用フェデレーション
    - https://meatwiki.nii.ac.jp/confluence/display/GakuNinShibInstall/signer
* テストフェデレーション
    - https://www.gakunin.jp/join/test/rule

In [ ]:
if 'federation' in gvars:
    !ansible {ugroup_name}_{manager_name} -a 'chdir={{{{base_dir}}}}/certs openssl x509 -in gakunin-signer.cer -sha256 -fingerprint -noout'

auth-proxy コンテナイメージを取得したことを確認します。


In [ ]:
!ansible {ugroup_name}_{manager_name} -m shell -a 'docker images | grep auth-proxy'

IdP-proxyを使用する場合は配置したIdP-proxyのサーバ証明書の内容を確認します。

In [ ]:
if 'idp_proxy_certificate_path' in gvars:
    !ansible {ugroup_name}_{manager_name} -a 'openssl x509 -in {{{{certs_dir}}}}/idp-proxy.cer -noout -text'

auth-proxyコンテナのためにセットアップしたディレクトリの状態を確認します。


In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'tree {{{{base_dir}}}}'

### JupyterHubの確認

インストールされたファイルを確認します。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'tree /srv/restuser'

restuserはホスト環境のサービスとして実行します。インストールしたrestuserサービスの状態を確認します。次のセルの出力結果にActive: active (running) と表示されることを確認してください。

In [ ]:
!ansible {ugroup_name}_{manager_name} -b -a 'systemctl status restuser'

`restuser`によってユーザ情報が取得できることを確認します。HTTPの応答がOK(200)となり、以下の情報が取得できることを確認してください。

```
{"name": "vcp", "dir": "/home/vcp", "shell": "/bin/bash", "uid": 1000, "gid": 1000}
```

In [ ]:
!ansible {ugroup_name}_{manager_name} -b -m shell \
    -a 'echo -e "POST /{{{{ansible_user}}}} HTTP/1.0\r\n" | nc -U /var/run/restuser.sock'

JupyterHubのコンテナイメージが取得できたことを確認します。


In [ ]:
!ansible {ugroup_name}_{manager_name} -a \
    'docker images -f label=org.jupyter.service=jupyterhub'

JupyterHubコンテナのために配置したファイルを確認します。


In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'tree {{{{jupyterhub_dir}}}}'

### PostgreSQLの確認

PostgreSQLコンテナのために配置したファイルを確認します。


In [ ]:
!ansible {ugroup_name}_{manager_name} -b -a 'tree {{{{postgres_dir}}}}'

### single-user Jupyter Notebook serverの確認

各VCノードのコンテナイメージ一覧を確認します。


In [ ]:
!ansible {ugroup_name} -m shell -a 'docker images | \
    grep -e "niicloudoperation/jupyterhub-singleuser"'

## コンテナの起動の確認

起動後の状態を確認します。

    コンテナが起動に失敗し何度も再起動されていないことを確認してください。



In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'docker stack ps {{{{ugroup_name}}}}'

postgres コンテナのログを表示します。


In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'docker service logs {{{{ugroup_name}}}}_postgres'

`jupyterhub` コンテナのログを表示します。

> PostgreSQLに接続できないなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'docker service logs {{{{ugroup_name}}}}_jupyterhub'

`auth-proxy` コンテナのログを表示します。

> 証明書設定に誤りがあるなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {ugroup_name}_{manager_name} -a 'docker service logs {{{{ugroup_name}}}}_auth-proxy'

## CoursewareHubにアクセスする

次のセルを実行すると、構築したCoursewareHubのアドレスを表示します。

In [ ]:
master_fqdn = gvars['master_fqdn']
print(f'https://{master_fqdn}')